In [1]:
from pathlib import Path
import xml.etree.ElementTree as ET
import numpy as np
import os
import os.path
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import uuid
import pandas as pd

In [24]:
def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(boxes.find("bndbox/ymin").text)/416
        xmin = int(boxes.find("bndbox/xmin").text)/416
        ymax = int(boxes.find("bndbox/ymax").text)/416
        xmax = int(boxes.find("bndbox/xmax").text)/416
        helmet = (boxes.find('name').text)

        width=(xmax-xmin)
        height=(ymax-ymin)
        x=xmin+width/2
        y=ymin+height/2

        if helmet=='helmet':
            flag=1
        else:
            flag=0

        list_with_single_boxes = [flag,x,y,width,height]
        list_with_all_boxes.append(list_with_single_boxes)

    return filename, list_with_all_boxes

In [29]:
df = pd.DataFrame(columns = ['image', 'text'])
for i in range(len(os.listdir('annotations'))):
    path=os.path.join('annotations/',os.listdir('annotations')[i])
    name, boxes = read_content(path)
    t=name.split('.')[0]
    filepath='yolov3/labels/'+t+".txt"
    np.savetxt(filepath,boxes,fmt='%i %.6f %.6f %.6f %.6f')
    df=df.append({'image':name,'text':filepath.split('/')[2]},ignore_index=True)